In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.datasets.dataset_metadata import NarrowbandMetadata
from torchsig.signals.builders.constellation import ConstellationSignalBuilder
from torchsig.signals.builders.tone import ToneSignalBuilder
import torchsig.transforms.functional as F
from torchsig.utils.dsp import frequency_shift

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
def generate_qpsk_signal(num_iq_samples: int = 128, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband QPSK Signal.

        Args:
        num_iq_samples (int, optional): Length of sample. Defaults to 10.
        scale (int, optional): scale normalized signal data. Defaults to 1.0.

        Returns:
            signal: generated Signal 

    """
    sample_rate = 10e6
    md = NarrowbandMetadata(
        num_iq_samples_dataset = num_iq_samples,
        fft_size = 4,
        impairment_level = 0,
        sample_rate = sample_rate,
        num_signals_min = 1,
        num_signals_distribution = [1.0],
        snr_db_min = 100.0,
        snr_db_max = 100.0,
        signal_duration_min = 1.00*num_iq_samples/sample_rate,
        signal_duration_max = 1.00*num_iq_samples/sample_rate,
        signal_bandwidth_min = sample_rate/4,
        signal_bandwidth_max = sample_rate/4,
        signal_center_freq_min = 0.0,
        signal_center_freq_max = 0.0,         
        class_list = ['qpsk'],
        class_distribution = [1.0],
        seed = 42
    )

    builder = ConstellationSignalBuilder(
        dataset_metadata = md, 
        class_name = 'qpsk',
        seed = 42
    )
    signal = builder.build()

    # normalize, then scale data   
    signal.data = F.normalize(
        data = signal.data,
        norm_order = 2,
        flatten = False
    )
    signal.data = np.multiply(signal.data, scale)

    return signal

In [ ]:
# test data
N = 1024
qpsk_data_baseband = generate_qpsk_signal(num_iq_samples = N, scale = 1.0).data
sample_rate = 1
cfo = sample_rate/4
qpsk_data_if = frequency_shift(qpsk_data_baseband,cfo,sample_rate)

# spectral inversion
impaired_qpsk_data = F.spectral_inversion(
    data = qpsk_data_if
)

freq_vec = np.arange(-1.0/2,1.0/2,1.0/(N))
qpsk_fft = np.fft.fftshift(np.fft.fft(qpsk_data_if))/np.sqrt(N)
impaired_qpsk_fft = np.fft.fftshift(np.fft.fft(impaired_qpsk_data))/np.sqrt(N)

fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1,1,1)
ax.plot(freq_vec, 20*np.log10(np.abs(qpsk_fft)),label='Input');
ax.plot(freq_vec, 20*np.log10(np.abs(impaired_qpsk_fft)),alpha=0.5,label='Spectral Inversion');
ax.set_ylim([-70, 0])
ax.set_xlim([freq_vec[0],freq_vec[-1]])
ax.set_ylabel('Magnitude (log10)',fontsize='large');
ax.set_title('QPSK with/without Spectral Inversion')
ax.legend(fontsize='large', loc='upper left');
ax.grid()


plt.show()